In [1]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import warnings
warnings.filterwarnings("ignore")

In [32]:
train_path = "/content/drive/MyDrive/Colab Notebooks/CNN_Sample1/mnist_train.csv"
test_path = "/content/drive/MyDrive/Colab Notebooks/CNN_Sample1/mnist_test.csv"

In [30]:
def load_and_preprocess(path):
    data = pd.read_csv(path)
    data = data.values
    np.random.shuffle(data)                 # mantığa 28 28 her resmin pikseli 255 e bölerek normalize ediyoz
    x = data[:,1:].reshape(-1,28,28,1)/255.0  # burda her pikseli alt alta array çeviriyoruz tekrar bak 
    y = data[:,0].astype(np.int32) #labeller datanın ilk kolonda belirlilmiş bu değerleri alıyoruz
    y = to_categorical(y,num_classes= len(set(y)))  # 0-9 arası label isimleri katogire ediyoruz => [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] bunun -gibi bu => 4
    return x , y


In [ ]:
x_train, y_train = load_and_preprocess(train_path)
x_test, y_test = load_and_preprocess(test_path)


In [24]:
NumberOfClass = y_train.shape[1]    # kaç tane labelimiz var
# CNN

model = Sequential()
model.add(Conv2D(input_shape = (28,28,1),filters=16,kernel_size = (3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())


model.add(Conv2D(filters=64,kernel_size = (3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(filters=128,kernel_size = (3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(units=256))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(units=NumberOfClass))   # çıkış sayısı layer kadar olması lazım
model.add(Activation("softmax"))




In [25]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
hist = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=25,batch_size=4000)


In [13]:
# save weigth
model.save_weights("weight.h5")

In [15]:
import json
with open("/content/deneme.json","w") as f:
    json.dump(hist.history,f)